In [1]:
import re
import time
#import itertools
from itertools import combinations, permutations, product, cycle, islice
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import plotly.graph_objects as go
import plotly.express as px
from bs4 import BeautifulSoup as bs4
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By

from optimize_lineup import Optimized_Lineups
from fantasy_utils import Fantasy_Projections as fp
import fantasy_utils as fu
fp = fu.Fantasy_Projections()

In [2]:
proj_url = 'https://xdl.baseball.cbssports.com/stats/stats-main'
elig_url = 'https://xdl.baseball.cbssports.com/teams/eligibility/'

In [3]:
eligibility_url_dict = {"8 Grand Kids":'30', 'Brewbirds':'3', 'Charmer':'36', 'Dirty Birds':'41', "Harveys Wallbangers":'4', 'Lil Trump & the Ivanabees':'27', 'Lima Time!':'38', 
                        'Roid Ragers':'44', 'Trouble with the Curve':'1', 'Ugly Spuds':'29', 'Wiscompton Wu-Tang':'42', 'Young Guns':'45'}
position_priority = ['C', '2B', '3B', 'SS', 'OF', '1B', 'MI', 'CI', 'DH', 'SP', 'RP']

In [ ]:
driver = webdriver.Chrome("C:\\WebDriver\\bin\\chromedriver.exe")
driver.get('https://www.cbssports.com/login?master_product=150&xurl=https%3A%2F%2Fwww.cbssports.com%2Flogin')
search_box = driver.find_element_by_id('userid')
search_box.send_keys('gostros09')
search_box = driver.find_element_by_id('password')
search_box.send_keys('Segneri9A')
search_box.submit()
time.sleep(3) # Let the user actually see something!
driver.get(proj_url)

# This clicks the Teams filter and opens it
time.sleep(3)
driver.find_element(By.XPATH, "//li[@class=' fantasyTeams dropdown selected_arrow large opt ']").click()

# This chooses the All Teams filter
time.sleep(4)
driver.find_element(By.XPATH, "//a[@aria-label='All Teams ']").click()

# This clicks on the Timeframe pulldown menu
time.sleep(3)
driver.find_element(By.XPATH, "//div[@class='pageFilter subFilters dropdown large']").click()
# This chooses the Rest of Season option
time.sleep(3)
driver.find_element(By.XPATH, "//li[@value='/stats/stats-main/restofseason']").click()

time.sleep(1)
html = driver.page_source
soup = bs4(html, 'html.parser')
pd.read_html(str(soup.find_all('table')[1:25]))

In [39]:
html = driver.page_source
soup = bs4(html, 'html.parser')

In [50]:
driver.quit()

In [192]:
#driver.find_element(By.XPATH, "//a[@class='sc-1hq60lq-3 sc-1yvalmu-5 NutYB ciyBov']").click()
#driver.find_element(By.XPATH, "//ul[@class='sc-1hq60lq-1 sc-1yvalmu-1 haeRSb jkqOBN dropDownFilter']/li[5]").click()
#driver.find_element(By.XPATH, "//a[@class='sc-1hq60lq-3 sc-1yvalmu-5 sc-1yvalmu-6 hvtxhx ciyBov JLOPU']").click()

In [11]:
name_change = {"8 Gran...":'Madness', 'Brewbi...':'Brewbirds', 'Charme...':'Charmer', 'Dirty ...':'Dirty Birds', 'Harvey...':'Harvey', 'Lil Tr...':'Lil Trump', 
               'Lima T...':'Lima Time', 'Roid R...':'Roiders', 'Troubl...':'Trouble', 'Ugly S...':'Ugly Spuds', 'Wiscom...':'Wu Tang', 'Young ...':'Young Guns'}

In [12]:
team_list = ['Lima Time!', 'Ugly Spuds', 'Harveys Wallbangers', 'Roid Ragers', 'Charmer', 'Wiscompton Wu-Tang', '10 O\'Clock Madness', 'Brewbirds', 'Trouble With The Curve', 'Lil Trump & The Ivanabees', 'Dirty Birds', 'Young Guns'] 
stat_list = ['BA', 'HR', 'R', 'RBI', 'SB', 'ERA', 'Sv+Hld', 'SO', 'W', 'WHIP']

In [4]:
def clean_table(df, num):
    df = df.copy()
    if num % 2 == 0:
        df.columns = df.columns.droplevel().droplevel()  
    else:
        df.columns = df.columns.droplevel()
    
    df.drop(columns='Action', inplace=True)
    df = df.iloc[:df.index.stop-1]
    df['Team'] = df.Player.apply(lambda x: x.split()[-1])
    df['Pos'] = df.Player.apply(lambda x: x.split()[-3])
    df['Player'] = df.Player.apply(lambda x: ' '.join(x.split()[:-3]))
    df['Owner'] = df['Avail'].replace(name_change)
    df.loc[df['Rank']<=50, 'top_50'] = 1
    df.loc[df['Rank']<=100, 'top_100'] = 1
    df.loc[df['Rank']<=276, 'top_276'] = 1
    df.loc[df['Rank']>276, 'above_276'] = 1
    if num % 2 == 0:
        df.loc[:13, 'Start'] = 1
    else:
        df.loc[:8, 'Start'] = 1
        df.rename(columns={'K':'SO', 'BB':'BBa', 'H':'Ha'},inplace=True)
        df['Sv+Hld'] = df['S'] + df['HD']
        df['ER'] = df['INNs']*df['ERA']/9
    
    df.fillna(0,inplace=True)
    return df

In [5]:
def login():
    driver = webdriver.Chrome("C:\\WebDriver\\bin\\chromedriver.exe")
    driver.get('https://www.cbssports.com/login?master_product=150&xurl=https%3A%2F%2Fwww.cbssports.com%2Flogin')
    search_box = driver.find_element_by_id('userid')
    search_box.send_keys('gostros09')
    search_box = driver.find_element_by_id('password')
    search_box.send_keys('Segneri9A')
    search_box.submit()
    return driver

In [6]:
def process_eligibility(val, driver):
    driver.get(elig_url+val)
    time.sleep(2.2)
    html = driver.page_source
    soup = bs4(html, 'html.parser')
    elig = pd.read_html(str(soup.find_all('table')))[0]
    elig = elig.iloc[:-1,:]
    elig['Player'] = elig['Player'].apply(lambda x: x[:re.search(r'RP|SP|1B|2B|3B|SS|LF|CF|RF|DH|C\s', x).span()[0]])
    elig.fillna(0,inplace=True)
    elig.loc[:,1:] = elig.iloc[:,1:].astype('int32')
    return elig

In [7]:
def scrape_proj(driver):
    time.sleep(2)
    driver.get(proj_url)

    # This clicks the Teams filter and opens it
    time.sleep(3)
    driver.find_element(By.XPATH, "//li[@class=' fantasyTeams dropdown selected_arrow large opt ']").click()

    # This chooses the All Teams filter
    time.sleep(4)
    driver.find_element(By.XPATH, "//a[@aria-label='All Teams ']").click()

    # This clicks on the Timeframe pulldown menu
    time.sleep(3)
    driver.find_element(By.XPATH, "//div[@class='pageFilter subFilters dropdown large']").click()
    # This chooses the Rest of Season option
    time.sleep(3)
    driver.find_element(By.XPATH, "//li[@value='/stats/stats-main/restofseason']").click()

    time.sleep(1)
    html = driver.page_source
    soup = bs4(html, 'html.parser')
    return pd.read_html(str(soup.find_all('table')[1:25]))

In [8]:
def parse_projections_data(master):
    df = pd.DataFrame()
    for n in range(len(master)):
        df = pd.concat([df, clean_table(master[n], n)])
    
    df.fillna(0,inplace=True)
    df.rename(columns={'AVG':'BA', 'INNs':'IP'}, inplace=True)
    df.drop(columns=['1B', '2B', '3B'],inplace=True)
    df = df.reset_index()
    return df

In [9]:
driver = login()

In [10]:
elig = pd.DataFrame()

for key, val in eligibility_url_dict.items():
    print(key)
    elig = pd.concat([elig, process_eligibility(val, driver)])

8 Grand Kids


C:\Users\pddnh\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:719: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  indexer = self._get_setitem_indexer(key)


Brewbirds
Charmer
Dirty Birds
Harveys Wallbangers
Lil Trump & the Ivanabees
Lima Time!
Roid Ragers
Trouble with the Curve
Ugly Spuds
Wiscompton Wu-Tang
Young Guns


In [38]:
elig.to_csv('data\\eligibility.csv', index=False)

In [ ]:
master = scrape_proj(driver)

In [40]:
#driver.find_element(By.XPATH, "//div[@class='pageFilter subFilters dropdown large']").click()
# This chooses the Rest of Season option
#time.sleep(3)
#driver.find_element(By.XPATH, "//li[@value='/stats/stats-main/restofseason']").click()

#html = driver.page_source
#soup = bs4(html, 'html.parser')
master = pd.read_html(str(soup.find_all('table')[1:25]))

In [41]:
df = parse_projections_data(master)

In [42]:
df = df.merge(elig, on='Player', how='left')

In [43]:
h_mask = (df['Start']==1) & (df['AB']>0) #(df['Pos'].isin(['C', '1B', '2B', '3B', 'LF', 'CF', 'RF', 'SS', 'DH']))
hitters = (df['AB']>0) #(df['Pos'].isin(['C', '1B', '2B', '3B', 'LF', 'CF', 'RF', 'SS', 'DH']))
p_mask = (df['Start']==1) & (df['AB']==0) #(df['Pos'].isin(['SP', 'RP']))
pitchers = (df['AB']==0) #(df['Pos'].isin(['SP', 'RP']))

df.loc[hitters, 'DH'] = 5
df.loc[pitchers, 'P'] = 5
df['P'].fillna(0,inplace=True)

df.loc[(df['SP']>=5) | (df['RP']>=5) | (df['P']>=5), 'type'] = 'p'
df['type'].fillna('h',inplace=True)
df.loc[(df['Player']=='Shohei Ohtani') & (df['AB']>10), 'type'] = 'h'

In [44]:
lgBA = df[h_mask]['H'].sum()/df[h_mask]['AB'].sum()
df.loc[hitters, 'zlgBA'] = (df[hitters]['H'] - (df[hitters]['AB'] * lgBA))

lgERA = df[p_mask]['ER'].sum()/df[p_mask]['IP'].sum()*9
df.loc[pitchers, 'zlgERA'] = (((df[pitchers]['ER']*9) - (df[pitchers]['IP']*lgERA))*-1)

lgWHIP = (df[p_mask]['BBa'].sum()+df[p_mask]['Ha'].sum())/df[p_mask]['IP'].sum()
df.loc[pitchers, 'zlgWHIP'] = (((df.loc[pitchers]['Ha']+df.loc[pitchers]['BBa'])-(df.loc[pitchers]['IP']*lgWHIP))*-1)

In [45]:
q = df[h_mask][['HR', 'RBI', 'R', 'SB', 'H', 'AB']].describe().T[['mean', 'std']].T.to_dict()
q['BA'] = {'mean':df[h_mask]['H'].sum()/df[h_mask]['AB'].sum(), 'std':df[h_mask]['BA'].std()}
q.update(df[p_mask][['W', 'SO', 'Sv+Hld', 'IP', 'ER', 'BBa', 'Ha']].describe().T[['mean', 'std']].T.to_dict())
q['ERA'] = {'mean':(df[p_mask]['ER'].sum()/df[p_mask]['IP'].sum())*9, 'std':(df[p_mask]['ER']/df[p_mask]['IP']*9).std()}
q['WHIP'] = {'mean':(df[p_mask]['BBa'].sum()+df[p_mask]['Ha'].sum())/df[p_mask]['IP'].sum(), 'std':((df[p_mask]['BBa']+df[p_mask]['Ha'])/df[p_mask]['IP']).std()}
q['zlgBA'] = {'mean':df[h_mask]['zlgBA'].mean(), 'std':df[h_mask]['zlgBA'].std()}
q['zlgERA'] = {'mean':df[p_mask]['zlgERA'].mean(), 'std':df[p_mask]['zlgERA'].std()}
q['zlgWHIP'] = {'mean':df[p_mask]['zlgWHIP'].mean(), 'std':df[p_mask]['zlgWHIP'].std()}
q

{'HR': {'mean': 0.8160714285714286, 'std': 0.32836564203248797},
 'RBI': {'mean': 2.877976190476192, 'std': 0.7794075437088037},
 'R': {'mean': 2.9380952380952388, 'std': 0.74478703493169},
 'SB': {'mean': 0.38690476190476225, 'std': 0.36958619420542266},
 'H': {'mean': 5.28511904761905, 'std': 1.0688087207499677},
 'AB': {'mean': 18.94761904761905, 'std': 3.1307046590806835},
 'BA': {'mean': 0.2789331490324202, 'std': 0.024793517847035165},
 'W': {'mean': 0.39351851851851843, 'std': 0.24921302232223116},
 'SO': {'mean': 5.922222222222222, 'std': 2.903279915528776},
 'Sv+Hld': {'mean': 0.3277777777777778, 'std': 0.5798538864188248},
 'IP': {'mean': 5.585185185185189, 'std': 3.195727720930012},
 'ER': {'mean': 2.1212016460905354, 'std': 1.3799405643127123},
 'BBa': {'mean': 1.7194444444444452, 'std': 0.9187006604040249},
 'Ha': {'mean': 5.103703703703703, 'std': 3.1178091375082726},
 'ERA': {'mean': 3.418116710875331, 'std': 0.6127391844627702},
 'WHIP': {'mean': 1.2216511936339522, 'st

In [46]:
for stat in ['R', 'HR', 'RBI', 'SB', 'zlgBA']:
    df.loc[hitters, 'z'+stat] = df[hitters].apply(lambda row: fp.big_board(row, stat, q), axis=1)

df.loc[hitters & (df['zlgBA'].isna()), 'zlgBA'] = df['zlgBA'].min()-.01
df.loc[hitters & (df['zzlgBA'].isna()), 'zzlgBA'] = df['zzlgBA'].min()-.01
df.loc[hitters, 'BIGAAh'] = df[hitters]['zR'] + df[hitters]['zRBI'] + df[hitters]['zHR'] + df[hitters]['zSB'] + df[hitters]['zzlgBA']

In [47]:
for stat in ['W', 'SO', 'Sv+Hld', 'zlgERA', 'zlgWHIP']:
    df.loc[pitchers, 'z'+stat] = df[pitchers].apply(lambda row: fp.big_board(row, stat, q), axis=1)

df.loc[pitchers & (df['zzlgERA'].isna()), 'zzlgERA'] = df['zzlgERA'].min()-.01
df.loc[pitchers & (df['zzlgWHIP'].isna()), 'zzlgWHIP'] = df['zzlgWHIP'].min()-.01
df.loc[pitchers, 'BIGAAp'] = df[pitchers]['zW']+df[pitchers]['zSO']+df[pitchers]['zSv+Hld']+df[pitchers]['zzlgERA']+df[pitchers]['zzlgWHIP']

In [48]:
df['BIGAAh'].fillna(0,inplace=True)
df['BIGAAp'].fillna(0,inplace=True)
df['z'] = df['BIGAAh']+df['BIGAAp']

In [49]:
ohtani_idx = df[df['Player']=='Shohei Ohtani'].index
df.loc[ohtani_idx[0], 'z'] = df.loc[ohtani_idx[0],'BIGAAh'] + 0
df.loc[ohtani_idx[1], 'z'] = 0 + df.loc[ohtani_idx[1],'BIGAAp']
df.loc[ohtani_idx[1], ['AB', 'R', 'H', 'HR', 'RBI', 'BB', 'K', 'SB', 'CS', 'BA', 'OBP', 'SLG', 'zlgBA', 'zR', 'zHR', 'zRBI', 'zSB', 'zzlgBA', 'BIGAAh']] = df.loc[ohtani_idx[0]][['AB', 'R', 'H', 'HR', 'RBI', 'BB', 'K', 'SB', 'CS', 'BA', 'OBP', 'SLG', 'zlgBA', 'zR', 'zHR', 'zRBI', 'zSB', 'zzlgBA', 'BIGAAh']]
df = df.loc[df.index!=ohtani_idx[0]].copy()

In [50]:
df[df['Start']==1].groupby('Owner')['z'].sum().sort_values()

Owner
Ugly Spuds    -16.394005
Brewbirds     -12.379966
Lil Trump     -10.421999
Wu Tang        -7.576272
Young Guns     -6.258933
Charmer        -3.748569
Madness         1.907186
Dirty Birds     8.532547
Harvey          8.561772
Lima Time       9.102882
Trouble        12.483666
Roiders        16.364545
Name: z, dtype: float64

In [51]:
def stitch_positions(row):
    pos_code = row[position_priority+['P']]>=5
    return list(pos_code[pos_code].index)

In [52]:
df['all_pos'] = df.apply(lambda x: stitch_positions(x), axis=1)

In [53]:
df.to_csv('data/period_6_projections.csv')
#df.to_csv('data/period_6_ros_projections.csv')


In [33]:
class Optimized_Lineups:
    def __init__(self, owner, data):
        self.owner = owner
        self.data = data
        self.d = self.data[self.data['Owner']==owner][['Player','all_pos', 'z', 'type']].set_index('Player').to_dict(orient='index')      
        self.p_dict = {k:v for (k,v) in self.d.items() if 'p' in v['type']}
        self.h_dict = {k:v for (k,v) in self.d.items() if 'h' in v['type']}
        self.catchers = [k for k,v in self.h_dict.items() if 'C' in v['all_pos']]
        self.first_basemen = [k for k,v in self.h_dict.items() if '1B' in v['all_pos']]
        self.second_basemen = [k for k,v in self.h_dict.items() if '2B' in v['all_pos']]
        self.third_basemen = [k for k,v in self.h_dict.items() if '3B' in v['all_pos']]
        self.shortstops = [k for k,v in self.h_dict.items() if 'SS' in v['all_pos']]
        self.middles = [k for k,v in self.h_dict.items() if 'MI' in v['all_pos']]
        self.corners = [k for k,v in self.h_dict.items() if 'CI' in v['all_pos']]
        self.outfielders = [k for k,v in self.h_dict.items() if 'OF' in v['all_pos']]
        self.dhs = [k for k,v in self.h_dict.items() if 'DH' in v['all_pos']]
        self.ofs = [i for i in combinations(self.outfielders, 5)]
        self.dh2 = [i for i in combinations(self.dhs, 2)]
        
    def _make_pitcher_combos(self):
        self.pitcher_combos = [i for i in combinations(self.p_dict.keys(), 9)]
        self.pitcher_z_list = self._z_list(self.pitcher_combos, self.p_dict)
        self.pitcher_idx = np.argmax(self.pitcher_z_list)
        self.pitcher_optimized_z = self.pitcher_z_list[self.pitcher_idx]
        self.pitcher_optimized_lineup = self.pitcher_combos[self.pitcher_idx]
        return
    
    def _z_list(self, pos_combos, player_dict):
        sum_z = 0
        z_list = []
        for i in range(len(pos_combos)):
            for name in pos_combos[i]:
                #print(i, sum_z)
                sum_z += player_dict[name]['z']
            z_list.append(sum_z)
            sum_z = 0
        return z_list
    
    def _make_hitter_combos(self):
        inf = [i for i in product(self.catchers, self.first_basemen, self.second_basemen, self.shortstops, self.third_basemen, self.middles, self.corners, self.ofs)]
        _list = []
        for num in range(len(inf)):
            _list.append([item for item in inf[num] if type(item)!=tuple]+[item for sublist in inf[num] for item in sublist if type(sublist)==tuple])
        
        _list2 = [i for i in _list if len(set(i))==12]
        inf2 = [i for i in product(_list2, self.dh2)]
        
        _list3 = []
        for num in range(len(inf2)):
            _list3.append([list(i) for i in inf2[num] if type(i)!=tuple][0]+[list(i) for i in inf2[num] if type(i)==tuple][0])
    
        self.hitter_combos = [i for i in _list3 if len(set(i))==14]
        print(len(self.hitter_combos))
        self.hitter_z_list = self._z_list(self.hitter_combos, self.h_dict)
        print(len(self.hitter_z_list))
        self.hitter_idx = np.argmax(self.hitter_z_list)
        self.hitters_optimized_z = self.hitter_z_list[self.hitter_idx]
        self.hitters_optimized_lineup = self.hitter_combos[self.hitter_idx]
        return    
    

In [123]:
opt_dict = {}
for t in name_change.values():
    print(t)
    opt_dict[t] = Optimized_Lineups(t, df)
    opt_dict[t]._make_pitcher_combos()
    opt_dict[t]._make_hitter_combos()
    print(f'Offense: {opt_dict[t].hitters_optimized_z}')
    print(f'Defense: {opt_dict[t].pitcher_optimized_z}')
    print(f'Total z: {opt_dict[t].pitcher_optimized_z + opt_dict[t].hitters_optimized_z}')

Madness
2828280
2828280
Offense: 10.855197179055908
Defense: -2.493803401693371
Total z: 8.361393777362537
Brewbirds
29700
29700
Offense: 6.567927273071329
Defense: 4.361600922398694
Total z: 10.929528195470024
Charmer
40488
40488
Offense: 0.5372634747176015
Defense: 6.24263380918847
Total z: 6.779897283906071
Dirty Birds
454230
454230
Offense: 13.732752249896267
Defense: 3.2301499942148975
Total z: 16.962902244111163
Harvey
1608
1608
Offense: 7.16827665173904
Defense: -3.760752621614114
Total z: 3.407524030124926
Lil Trump
2790
2790
Offense: -14.008577204611631
Defense: 2.7672383846124573
Total z: -11.241338819999175
Lima Time
138360
138360
Offense: -9.703312800433354
Defense: 6.729937600074528
Total z: -2.9733752003588254
Roiders
2561328
2561328
Offense: 12.401831447413935
Defense: 10.24593344004044
Total z: 22.647764887454375
Trouble
1401204
1401204
Offense: -1.1092851019541046
Defense: 4.033443066606676
Total z: 2.9241579646525713
Ugly Spuds
23760
23760
Offense: -0.2950929771375059

In [56]:
#df = pd.read_csv('optimized_lineup_data.csv')

In [22]:
driver.quit()

NameError: name 'driver' is not defined

In [492]:
df.to_csv('optimized_lineup_data2.csv')

In [23]:
position_priority = ['C', '2B', '3B', 'SS', 'OF', '1B', 'MI', 'CI', 'DH', 'SP', 'RP']
optimized = {
    'Madness':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Brewbirds':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Charmer':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Dirty Birds':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Harvey':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Lil Trump':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Lima Time':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Roiders':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Trouble':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Ugly Spuds':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Wu Tang':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Young Guns':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
            }
optimized

{'Madness': {'C': [-1],
  '2B': [-1],
  '3B': [-1],
  'OF': [-1, -1, -1, -1, -1],
  'SS': [-1],
  '1B': [-1],
  'MI': [-1],
  'CI': [-1],
  'DH': [-1, -1],
  'SP': [-1, -1, -1, -1, -1, -1, -1, -1, -1],
  'RP': [-1, -1, -1, -1, -1, -1],
  'P': [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
  'B': [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]},
 'Brewbirds': {'C': [-1],
  '2B': [-1],
  '3B': [-1],
  'OF': [-1, -1, -1, -1, -1],
  'SS': [-1],
  '1B': [-1],
  'MI': [-1],
  'CI': [-1],
  'DH': [-1, -1],
  'SP': [-1, -1, -1, -1, -1, -1, -1, -1, -1],
  'RP': [-1, -1, -1, -1, -1, -1],
  'P': [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
  'B': [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]},
 'Charmer': {'C': [-1],
  '2B': [-1],
  '3B': [-1],
  'OF': [-1, -1, -1, -1, -1],
  'SS': [-1],
  '1B': [-1],
  'MI': [-1],
  'CI': [-1],
  'DH': [-1, -1],
  'SP': [-1, -1, -1, -1, -1, -1, -1, -1, -1],
  'RP': [-1, -1, -1, -1, -1, -1],
  'P': [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
  'B': [-1, -1, -1, -1, -1, -1, -1, -1, -1,

In [24]:
for tm in df.Owner.unique():
    df.loc[df['Owner']==tm, 'opt'] = 'B'
    for df_idx, row in df[df['Owner']==tm].sort_values('z', ascending=False).iterrows():
        pos_code = df.loc[df_idx][position_priority+['P']]>=5
        #print(df_idx, row['Player'], pos_code[pos_code].index[0])

        for player_pos_val in pos_code[pos_code].index: #position list like ['SP', 'RP']
            if df.loc[df_idx, 'opt'] == 'B':
                for optimized_pos_idx in range(len(optimized[tm][player_pos_val])):
                    if optimized[tm][player_pos_val][optimized_pos_idx] == -1:
                        optimized[tm][player_pos_val][optimized_pos_idx] = row['Player']
                        df.loc[df_idx, 'opt'] = player_pos_val
                        break
                    #else: tries the next position in the optimizer position list        


In [25]:
pd.DataFrame(optimized)

,Madness,Brewbirds,Charmer,Dirty Birds,Harvey,Lil Trump,Lima Time,Roiders,Trouble,Ugly Spuds,Wu Tang,Young Guns
C,[Salvador Perez],[Sean Murphy],[Shea Langeliers],[Daulton Varsho],[William Contreras],[Danny Jansen],[Adley Rutschman],[Willson Contreras],[Alejandro Kirk],[Will Smith],[Cal Raleigh],[Travis d'Arnaud]
2B,[Andres Gimenez],[Whit Merrifield],[Mookie Betts],[Jeff McNeil],[Jazz Chisholm],[Brendan Donovan],[Jose Altuve],[Luis Arraez],[Marcus Semien],[Isaac Paredes],[Kolten Wong],[Tommy Edman]
3B,[Ke'Bryan Hayes],[Austin Riley],[Jose Ramirez],[Nolan Arenado],[Rafael Devers],[Gunnar Henderson],[Ryan McMahon],[Ty France],[Bobby Witt],[Alex Bregman],[Manny Machado],[Matt Chapman]
OF,"[Kris Bryant, Adolis Garcia, Brandon Nimmo, St...","[Juan Soto, Michael Harris, Jake Fraley, Randa...","[Bryan Reynolds, Giancarlo Stanton, Seiya Suzu...","[Starling Marte, Taylor Ward, Thairo Estrada, ...","[Julio Rodriguez, Kyle Schwarber, Corbin Carro...","[Byron Buxton, Eloy Jimenez, Teoscar Hernandez...","[Kyle Tucker, Tyler O'Neill, Josh Naylor, Jake...","[Aaron Judge, Joc Pederson, Charlie Blackmon, ...","[George Springer, Randy Arozarena, Mitch Hanig...","[Nick Castellanos, Christian Yelich, Oscar Gon...","[Mike Trout, Luis Robert, Ronald Acuna, Ramon ...","[Yordan Alvarez, Cedric Mullins, Andrew Vaughn..."
SS,[Dansby Swanson],[Willy Adames],[Oswald Peraza],[Tim Anderson],[Amed Rosario],[Francisco Lindor],[Xander Bogaerts],[Javier Baez],[Ezequiel Tovar],[Bo Bichette],[Nico Hoerner],[Trea Turner]
1B,[Paul Goldschmidt],[Freddie Freeman],[Pete Alonso],[Anthony Rizzo],[Jose Abreu],[Matt Olson],[Vinnie Pasquantino],[Vladimir Guerrero],[Christian Walker],[C.J. Cron],[Nate Lowe],[Jake Cronenworth]
MI,[Ozzie Albies],[Gleyber Torres],[Ketel Marte],[Wander Franco],[Jonathan India],[C.J. Abrams],[Max Muncy],[Jeremy Pena],[Josh Rojas],[Corey Seager],[Jean Segura],[Carlos Correa]
CI,[Rhys Hoskins],[Justin Turner],[Ryan Mountcastle],[Josh Bell],[Triston Casas],[Rowdy Tellez],[Alec Bohm],[Seth Brown],[Lourdes Gurriel],[Jose Miranda],[-1],[Eugenio Suarez]
DH,"[Jorge Polanco, Tyler Stephenson]","[Vaughn Grissom, Michael Conforto]","[Shohei Ohtani, Esteury Ruiz]","[J.T. Realmuto, Brandon Drury]","[Fernando Tatis, Matt Mervis]","[Juan Yepez, Lars Nootbaar]","[Christopher Morel, Andrew Benintendi]","[Brandon Lowe, Trent Grisham]","[J.D. Martinez, Eddie Rosario]","[Oneil Cruz, Mike Yastrzemski]","[Masataka Yoshida, Alek Thomas]","[Luis Urias, Jordan Walker]"
SP,"[Clayton Kershaw, Zac Gallen, Alex Cobb, Aaron...","[Max Scherzer, Aaron Nola, Robbie Ray, Logan W...","[Jose Urquidy, Sonny Gray, Luis Garcia, Joe Ry...","[Kevin Gausman, Joe Musgrove, Jesus Luzardo, P...","[Nestor Cortes, Tony Gonsolin, Dustin May, Kyl...","[Jacob deGrom, Gerrit Cole, Zach Eflin, Tristo...","[Cristian Javier, Justin Verlander, Shane McCl...","[Julio Urias, Carlos Rodon, Zack Wheeler, Max ...","[Dylan Cease, Corbin Burnes, Freddy Peralta, R...","[Brandon Woodruff, Yu Darvish, Luis Castillo, ...","[Sandy Alcantara, Spencer Strider, Chris Bassi...","[Alek Manoah, Luis Severino, Nathan Eovaldi, M..."


In [124]:
for t in name_change.values():
    print(f'{t} z: {opt_dict[t].hitters_optimized_z + opt_dict[t].pitcher_optimized_z}')
    df.loc[df['Player'].isin(opt_dict[t].hitters_optimized_lineup), 'opt'] = 1
    df.loc[df['Player'].isin(opt_dict[t].pitcher_optimized_lineup), 'opt'] = 1

Madness z: 8.361393777362537
Brewbirds z: 10.929528195470024
Charmer z: 6.779897283906071
Dirty Birds z: 16.962902244111163
Harvey z: 3.407524030124926
Lil Trump z: -11.241338819999175
Lima Time z: -2.9733752003588254
Roiders z: 22.647764887454375
Trouble z: 2.9241579646525713
Ugly Spuds z: 8.94818643401536
Wu Tang z: 3.789802890999136
Young Guns z: 2.5485312755846734


In [125]:
def total_stats(row):
    #return row['H'].sum() / row['AB'].sum(), row['HR'].sum(), row['R'].sum(), row['RBI'].sum(), row['SB'].sum()
    return row['W'].sum(), row['Sv+Hld'].sum(), row['SO'].sum(), round(row['ER'].sum()/row['IP'].sum()*9,2), round((row['Ha'].sum()+row['BBa'].sum())/row['IP'].sum(),2)

In [126]:
tm_proj = df[df['opt']==1].groupby('Owner')[['H', 'AB', 'Ha', 'BBa', 'ER', 'IP', 'R', 'RBI', 'HR', 'SB', 'W', 'SO', 'Sv+Hld']].sum()#.apply(lambda x: total_stats(x))
tm_proj['BA'] = tm_proj['H']/tm_proj['AB']
tm_proj['ERA'] = round(tm_proj['ER']/tm_proj['IP']*9,2)
tm_proj['WHIP'] = round((tm_proj['Ha']+tm_proj['BBa'])/tm_proj['IP'],2)
tm_proj

,H,AB,Ha,BBa,ER,IP,R,RBI,HR,SB,W,SO,Sv+Hld,BA,ERA,WHIP
Owner,,,,,,,,,,,,,,,,
Brewbirds,1958.0,7033.0,989.0,320.0,407.876667,1165.0,1088.0,1054.0,302.0,145.0,76.0,1292.0,105.0,0.278402,3.15,1.12
Charmer,2022.0,7373.0,839.0,297.0,345.966667,1047.0,1104.0,1169.0,361.0,123.0,72.0,1147.0,108.0,0.274244,2.97,1.09
Dirty Birds,2098.0,7301.0,898.0,318.0,362.004444,1086.0,1132.0,1045.0,287.0,152.0,60.0,1203.0,127.0,0.287358,3.00,1.12
Harvey,1893.0,6985.0,1128.0,395.0,491.978889,1330.0,1096.0,1048.0,342.0,148.0,98.0,1286.0,0.0,0.271009,3.33,1.15
Lil Trump,1807.0,6742.0,863.0,291.0,389.786667,1070.0,1000.0,1028.0,308.0,77.0,65.0,1222.0,124.0,0.268021,3.28,1.08
Lima Time,1874.0,6714.0,952.0,322.0,398.198889,1159.0,986.0,978.0,275.0,118.0,87.0,1296.0,57.0,0.279118,3.09,1.10
Madness,2048.0,7192.0,981.0,343.0,420.094444,1156.0,1086.0,1037.0,285.0,171.0,69.0,1221.0,111.0,0.284761,3.27,1.15
Roiders,2038.0,7187.0,983.0,314.0,391.123333,1171.0,1096.0,1123.0,347.0,83.0,83.0,1297.0,105.0,0.283568,3.01,1.11
Trouble,1956.0,7205.0,1029.0,357.0,431.970000,1244.0,1043.0,1046.0,302.0,145.0,78.0,1294.0,69.0,0.271478,3.13,1.11


In [129]:
for s in stat_list:
    #print(s)
    tm_proj[s+'_rank'] = tm_proj[s].rank()

tm_proj['Pts_rank'] = tm_proj.loc[:,tm_proj.columns.str.contains('_rank')].sum(axis=1)

In [130]:
tm_proj.loc[:,tm_proj.columns.str.contains('_rank')].sort_values('Pts_rank', ascending=False)

,BA_rank,HR_rank,R_rank,RBI_rank,SB_rank,ERA_rank,Sv+Hld_rank,SO_rank,W_rank,WHIP_rank,Pts_rank
Owner,,,,,,,,,,,
Harvey,2.0,10.0,9.5,8.0,9.0,12.0,1.0,6.0,11.0,11.0,79.5
Roiders,10.0,11.0,9.5,11.0,2.0,3.0,6.5,10.5,8.5,5.5,77.5
Wu Tang,7.0,4.0,3.0,1.0,11.0,11.0,3.0,12.0,12.0,11.0,75.0
Brewbirds,8.0,6.5,7.5,9.0,7.5,7.5,6.5,7.0,6.0,8.5,74.0
Madness,11.0,2.0,6.0,5.0,12.0,9.0,9.0,4.0,4.0,11.0,73.0
Dirty Birds,12.0,3.0,12.0,6.0,10.0,2.0,11.0,3.0,1.0,8.5,68.5
Ugly Spuds,5.0,9.0,5.0,10.0,3.0,4.0,5.0,10.5,8.5,5.5,65.5
Charmer,4.0,12.0,11.0,12.0,5.0,1.0,8.0,1.5,5.0,2.0,61.5
Trouble,3.0,6.5,4.0,7.0,7.5,6.0,4.0,8.0,7.0,5.5,58.5


In [111]:
ol = Optimized_Lineups('Lima Time', df)
ol._make_pitcher_combos()
print(len(ol.pitcher_combos))
#ol.catchers
ol._make_hitter_combos()

11440
138360
138360


In [112]:
ol.pitcher_optimized_z, ol.pitcher_optimized_lineup

(6.729937600074528,
 ('Justin Verlander',
  'Shane McClanahan',
  'Cristian Javier',
  'Charlie Morton',
  'Lucas Giolito',
  'Carlos Carrasco',
  'Erik Swanson',
  'Jason Adam',
  'A.J. Minter'))

In [113]:
ol.hitters_optimized_z, ol.hitters_optimized_lineup

(-9.703312800433354,
 ['Adley Rutschman',
  'Vinnie Pasquantino',
  'Jose Altuve',
  'Xander Bogaerts',
  'Alec Bohm',
  'Ryan McMahon',
  'Darick Hall',
  'Kyle Tucker',
  "Tyler O'Neill",
  'Alex Verdugo',
  'Josh Naylor',
  'Andrew Benintendi',
  'Max Muncy',
  'Jake McCarthy'])

In [115]:
(df[df['Owner']=='Harvey'][position_priority]>=5).sum()

C      1
2B     2
3B     3
SS     1
OF     8
1B     3
MI     3
CI     5
DH    16
SP    10
RP     4
dtype: int64

In [127]:
df[df['Player'].isin(list(hitter_combos[np.argmax(hitter_z_list)]))][['BA', 'HR', 'SB', 'R', 'RBI']]#.sum()

NameError: name 'hitter_combos' is not defined

In [117]:
df[df['Player'].isin(list(pitcher_combos[np.argmax(pitcher_z_list)]))][['ERA', 'WHIP', 'W', 'SO', 'Sv+Hld']]#.sum()

NameError: name 'pitcher_combos' is not defined

In [44]:
hitter_z_list[np.argmax(hitter_z_list)]+pitcher_z_list[np.argmax(pitcher_z_list)]

-7.689999999999998

In [118]:
df[(df['Owner']=='Lima Time') & (df['Start']==1)]
df[(df['Owner']=='Lima Time')][['Player', 'z', 'Sv+Hld']]#.z.sum()

,Player,z,Sv+Hld
199,Kyle Tucker,3.659651,0.0
200,Adley Rutschman,-3.413719,0.0
201,Xander Bogaerts,2.584603,0.0
202,Vinnie Pasquantino,-2.401900,0.0
203,Jose Altuve,-2.825672,0.0
204,Tyler O'Neill,2.865511,0.0
205,Max Muncy,-1.735309,0.0
206,Jake McCarthy,-0.559118,0.0
207,MJ Melendez,-5.638751,0.0
208,Alec Bohm,-2.470757,0.0


In [119]:
ranks = df.groupby('Owner').agg({'top_50':'sum', 'top_100':'sum', 'top_276':'sum', 'above_276':'sum'}).sort_values('top_276', ascending=False).reset_index()
ranks['avg'] = ranks.iloc[:,1:4].mean(axis=1)
ranks

,Owner,top_50,top_100,top_276,above_276,avg
0,Dirty Birds,4.0,10.0,24.0,9.0,12.666667
1,Lima Time,3.0,8.0,24.0,9.0,11.666667
2,Brewbirds,6.0,8.0,23.0,10.0,12.333333
3,Harvey,5.0,7.0,23.0,10.0,11.666667
4,Trouble,4.0,8.0,23.0,10.0,11.666667
5,Wu Tang,7.0,9.0,23.0,10.0,13.000000
6,Young Guns,3.0,8.0,23.0,10.0,11.333333
7,Ugly Spuds,3.0,9.0,22.0,11.0,11.333333
8,Roiders,3.0,8.0,21.0,12.0,10.666667
9,Charmer,4.0,5.0,20.0,13.0,9.666667


In [120]:
stats = df[(df['Start']==1) & (df['Pos'].isin(['1B', '2B', '3B', 'SS', 'C', 'OF', 'DH']))].groupby('Owner')[['H', 'AB', 'HR', 'SB', 'R', 'RBI', 'W', 'K', 'Sv+Hld']].sum().sort_values('K', ascending=False)
stats['BA'] = stats['H']/stats['AB']

In [121]:
stats

,H,AB,HR,SB,R,RBI,W,K,Sv+Hld,BA
Owner,,,,,,,,,,
Young Guns,1598.0,5842.0,243.0,108.0,896.0,834.0,0.0,1412.0,0.0,0.273536
Harvey,1368.0,4971.0,233.0,97.0,769.0,753.0,0.0,1211.0,0.0,0.275196
Roiders,1454.0,5115.0,231.0,58.0,778.0,762.0,0.0,1128.0,0.0,0.284262
Madness,1327.0,4699.0,200.0,93.0,673.0,716.0,0.0,1105.0,0.0,0.282401
Brewbirds,1307.0,4661.0,186.0,98.0,695.0,694.0,0.0,1040.0,0.0,0.280412
Charmer,1242.0,4519.0,229.0,68.0,654.0,779.0,14.0,1012.0,0.0,0.274840
Dirty Birds,1482.0,5169.0,196.0,92.0,762.0,734.0,0.0,934.0,0.0,0.286709
Ugly Spuds,1112.0,4004.0,192.0,47.0,596.0,656.0,0.0,899.0,0.0,0.277722
Lima Time,1071.0,3966.0,172.0,44.0,576.0,560.0,0.0,888.0,0.0,0.270045
